In [1]:
import asyncio
import os
from pydantic import BaseModel
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

In [2]:
class OutputStruct(BaseModel):
    """A structured output for testing purposes."""

    city: str
    description: str

In [3]:
print("=== Non-streaming example ===")

credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    deployment_name=os.environ["AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"],
    credential=credential,
)


agent = client.as_agent(
    name="CityAgent",
    instructions="You are a helpful agent that describes cities in a structured format.",
)

=== Non-streaming example ===


In [4]:
query = "Tell me about Paris, France"
print(f"User: {query}")

User: Tell me about Paris, France


In [5]:
result = await agent.run(query, options={"response_format": OutputStruct})

if structured_data := result.value:
    print("Structured Output Agent:")
    print(f"City: {structured_data.city}")
    print(f"Description: {structured_data.description}")
else:
    print(f"Failed to parse response: {result.text}")

Structured Output Agent:
City: Paris
Description: Paris is the capital city of France, known for its art, fashion, culture, and gastronomy. It is home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city is also famous for its charming streets, cafes, and vibrant atmosphere, making it a leading global city.


In [6]:
print("=== Streaming example ===")

agent = client.as_agent(
    name="CityAgent",
    instructions="You are a helpful agent that describes cities in a structured format.",
)

query = "Tell me about Tokyo, Japan"
print(f"User: {query}")

=== Streaming example ===
User: Tell me about Tokyo, Japan


In [7]:
stream = agent.run(query, stream=True, options={"response_format": OutputStruct})
async for update in stream:
    if update.text:
        print(update.text, end="", flush=True)
print()

{"city":"Tokyo","description":"Tokyo, the capital of Japan, is a bustling metropolis known for its blend of traditional and modern architecture, including historic temples and cutting-edge skyscrapers. It is a global financial center, rich in culture, with numerous museums, parks, and shopping districts. Renowned for its food scene, Tokyo offers everything from sushi and ramen to international cuisine. The city is also famous for its efficient public transportation, anime culture, and vibrant nightlife."}


In [8]:
result = await stream.get_final_response()

if structured_data := result.value:
    print("Structured Output (from streaming with ResponseStream):")
    print(f"City: {structured_data.city}")
    print(f"Description: {structured_data.description}")
else:
    print(f"Failed to parse response: {result.text}")

Structured Output (from streaming with ResponseStream):
City: Tokyo
Description: Tokyo, the capital of Japan, is a bustling metropolis known for its blend of traditional and modern architecture, including historic temples and cutting-edge skyscrapers. It is a global financial center, rich in culture, with numerous museums, parks, and shopping districts. Renowned for its food scene, Tokyo offers everything from sushi and ramen to international cuisine. The city is also famous for its efficient public transportation, anime culture, and vibrant nightlife.
